In [117]:
import cv2 as cv
import tensorflow.keras as kr

In [118]:
(X_train, y_train), (X_test, y_test) = kr.datasets.mnist.load_data()

In [119]:
X_train.shape

(60000, 28, 28)

In [120]:
(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)

(60000, 28, 28, 1)

In [121]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)

In [122]:
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

In [123]:
input_shape = (X_test.shape[1], X_test.shape[2], 1)

In [124]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [125]:
X_train /= 255
X_test /= 255

In [126]:
import pandas as pd

In [127]:
y_train = pd.get_dummies(y_train)

In [128]:
y_test = pd.get_dummies(y_test)

In [129]:
kr.losses.sparse_categorical_crossentropy

<function keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1, ignore_class=None)>

In [131]:
model = kr.models.Sequential()

model.add(kr.layers.Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal', activation = kr.activations.relu, input_shape = (28,28,1)))
model.add(kr.layers.MaxPool2D(2))
model.add(kr.layers.BatchNormalization())

model.add(kr.layers.Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal', activation = kr.activations.relu))
model.add(kr.layers.MaxPool2D(2))
model.add(kr.layers.BatchNormalization())

model.add(kr.layers.Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal', activation = kr.activations.relu))
model.add(kr.layers.MaxPool2D(2))
model.add(kr.layers.BatchNormalization())

model.add(kr.layers.Flatten())
model.add(kr.layers.Dropout(0.3))

model.add(kr.layers.Dense(128, kernel_initializer = 'he_normal', activation = kr.activations.relu))
model.add(kr.layers.Dropout(0.2))
model.add(kr.layers.Dense(256, kernel_initializer = 'he_normal', activation = kr.activations.relu))
model.add(kr.layers.Dropout(0.2))
model.add(kr.layers.Dense(10, activation = 'softmax'))

model.compile(optimizer = kr.optimizers.Adam(learning_rate = 3e-4), loss = kr.losses.categorical_crossentropy, metrics = ['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 14, 14, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                        

In [135]:
early = kr.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15)

In [136]:
model.fit(X_train, y_train, batch_size=32, validation_data=(X_test, y_test), callbacks=[early], epochs=200)

Epoch 1/200
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1247 - accuracy: 0.9621 - val_loss: 0.0452 - val_accuracy: 0.9857
Epoch 2/200
1875/1875 [==============================] - 16s 9ms/step - loss: 0.0729 - accuracy: 0.9783 - val_loss: 0.0407 - val_accuracy: 0.9874
Epoch 3/200
1875/1875 [==============================] - 16s 9ms/step - loss: 0.0551 - accuracy: 0.9832 - val_loss: 0.0299 - val_accuracy: 0.9905
Epoch 4/200
1875/1875 [==============================] - 16s 9ms/step - loss: 0.0437 - accuracy: 0.9868 - val_loss: 0.0349 - val_accuracy: 0.9895
Epoch 5/200
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0397 - accuracy: 0.9882 - val_loss: 0.0317 - val_accuracy: 0.9899
Epoch 6/200
1875/1875 [==============================] - 16s 9ms/step - loss: 0.0305 - accuracy: 0.9906 - val_loss: 0.0331 - val_accuracy: 0.9907
Epoch 7/200
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0272 - accuracy: 0.9915 - val_loss: 0.0277 -

In [137]:
y_hatx = model.predict(X_test)

313/313 [==============================] - 1s 4ms/step


In [138]:
y_hatx.shape

(10000, 10)

In [139]:
y_hat = [y_hatx[i].argmax() for i in range(y_hatx.shape[0])]

In [140]:
y_true = y_test.apply(lambda x: x.argmax(), axis=1)

In [141]:
(y_hat == y_true).mean()*100

99.33

In [142]:
from sklearn.metrics import classification_report, confusion_matrix

In [143]:
print(classification_report(y_true, y_hat))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       980
           1       0.99      1.00      1.00      1135
           2       1.00      0.99      1.00      1032
           3       0.99      1.00      0.99      1010
           4       0.99      1.00      0.99       982
           5       0.99      0.99      0.99       892
           6       1.00      0.99      0.99       958
           7       0.99      0.99      0.99      1028
           8       0.99      0.99      0.99       974
           9       1.00      0.99      0.99      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000



In [144]:
print(confusion_matrix(y_true, y_hat))

[[ 978    0    0    0    0    0    1    1    0    0]
 [   0 1132    0    1    0    2    0    0    0    0]
 [   1    0 1025    0    0    0    0    4    2    0]
 [   0    0    1 1005    0    3    0    0    1    0]
 [   0    0    0    0  978    0    0    0    1    3]
 [   1    0    0    6    0  882    1    1    0    1]
 [   4    3    0    0    1    1  947    0    2    0]
 [   0    4    1    0    1    0    0 1022    0    0]
 [   2    1    1    1    0    0    0    0  968    1]
 [   0    0    0    0    6    2    0    3    2  996]]


In [145]:
model.save('mnistNew.h5', save_format='h5')